In [ ]:
from keras.datasets import mnist
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#libraries

In [ ]:
(x , y) , (x_test , y_test) = mnist.load_data()
#load dataset

In [ ]:
#Standardize dataset
x = x.reshape(x.shape[0] ,-1 )
x_test = x_test.reshape(x_test.shape[0] ,-1 )
x_test = (x_test - np.mean(x)) / np.std(x)
x = (x - np.mean(x)) / np.std(x)

In [ ]:
#Divide data into training and test
x_train,x_valid,y_train,y_valid=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#4-Apply one hot vector for labels (meaning the value is 1 in the correct class and 0 in the rest, there will be 10 classes so a vector of 10).
num_of_calsses=10
y_train_onehot = np.zeros((y_train.shape[0], num_of_calsses))
y_train_onehot[np.arange(y_train.shape[0]), y_train] = 1
y_valid_onehot = np.zeros((y_valid.shape[0], num_of_calsses))
y_valid_onehot[np.arange(y_valid.shape[0]), y_valid] = 1

In [ ]:
def initial_weight(prev, curr, val):
  if val == 0:
    return np.random.uniform(-0.5,0.5,(784,curr))
  else:
     return np.random.uniform(-0.5,0.5,(prev, curr))
def sigmoid(z):
  return 1/(1+np.exp(-z))
def soft_max(z):
  return np.exp(z)/np.sum(np.exp(z))
def mse(y_true,y_pred):
  return (np.mean(y_true-y_pred)**2)
def forward_pass(X, weights):
  num_layers = len(weights) + 1
  layer_outputs = []
  input_data = X
  for i in range(num_layers - 1):
    weight_matrix = weights[i]
    layer_input = np.dot(input_data, weight_matrix)
    layer_output = sigmoid(layer_input)
    layer_outputs.append(layer_output)
    input_data = layer_output
  return layer_outputs

def backpropagation(X, y, layer_outputs, weights, learning_rate=.1):
  num_layers = len(weights)
  num_samples = X.shape[0]
  layer_outputs.reverse()
  error = (layer_outputs[0] - y) * layer_outputs[0] * (1 - layer_outputs[0])
  for i in range(num_layers):
    if i==0:
      continue
    gradient = np.dot(layer_outputs[i].T, error)
    weights[i]-=(learning_rate/num_samples)*gradient #update weights
    error=np.dot(error, weights[i].T)* layer_outputs[i] * (1 - layer_outputs[i])
  return weights


In [ ]:
class NN:
  def __init__(self, x, y, num_of_layers, size_of_layers):
    self.X = x
    self.y = y
    self.num_of_layers = num_of_layers
    self.size_of_layers = size_of_layers
    self.weights = []

  def fit(self, epochs, learning_rate):
    prev_layer_size = 0
    for i in range(self.num_of_layers):
      curr_layer_size = self.size_of_layers[i]
      layer_weights = initial_weight(prev_layer_size, curr_layer_size, i)
      self.weights.append(layer_weights)
      prev_layer_size = curr_layer_size


    for epoch in range(epochs):
      layer_outputs = forward_pass(self.X, self.weights)
      loss = mse(self.y, layer_outputs[-1])
      # if epoch%100==0:
      #   print(f"Epoch {epoch} Loss: {loss}")
      self.weights = backpropagation(self.X, self.y, layer_outputs, self.weights, learning_rate)

  def predict(self, X_test):
    layer_outputs = forward_pass(X_test, self.weights)
    predictions = np.argmax(layer_outputs[-1], axis=1)#get max argument
    return predictions
nn = NN(x_train, y_train_onehot, num_of_layers=2, size_of_layers=[64, 10])

nn.fit(epochs=1000, learning_rate=.1)
predictions = nn.predict(x_valid)
accuracy = np.mean(predictions == y_valid)
print(f"Accuracy ={accuracy*100}%")


Epoch 0 Loss: 0.16230520587505107
Epoch 100 Loss: 0.000280524497357473
Epoch 200 Loss: 0.0006781638027781421
Epoch 300 Loss: 0.0006322388475877101
Epoch 400 Loss: 0.0004484719196212251
Epoch 500 Loss: 0.00025341464971223506
Epoch 600 Loss: 0.00012124447627058834
Epoch 700 Loss: 4.88385334648977e-05
Epoch 800 Loss: 1.2486219905325688e-05
Epoch 900 Loss: 2.3850681955177466e-07
Accuracy =43.766666666666666%


In [ ]:
#second example  Build NN with 3 layers => 2 hidden layers
nn1 = NN(x_train, y_train_onehot, num_of_layers=3, size_of_layers=[20,10, 10])
nn1.fit(epochs=1000, learning_rate=.1)
predictions = nn1.predict(x_valid)
accuracy = np.mean(predictions == y_valid)
print(f"Accuracy ={accuracy*100}%")

Accuracy =53.475902%


In [ ]:
#third example Build NN with 3 layers=> 2 hidden layer
nn2 = NN(x_train, y_train_onehot, num_of_layers=3, size_of_layers=[20,30, 10])
nn2.fit(epochs=1000, learning_rate=.1)
predictions = nn2.predict(x_valid)
accuracy = np.mean(predictions == y_valid)
print(f"Accuracy ={accuracy*100}%")

Accuracy =54.6784125902%
